In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/enron-pair/enron-feng.csv


In [2]:
!pip3 install transformers

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
!pip3 install "numpy>=1.16.5,<1.23.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
cuml 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
cudf 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
raft-dask 23.4.0 requires dask==2023.3.2, but you have dask 2023.4.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you

In [5]:
df = pd.read_csv('/kaggle/input/enron-pair/enron-feng.csv')
df.head(5)

,Unnamed: 0,Received,Response
0,0,resumes of whom ?,The commercial support people that you and Hun...
1,1,Christy I read these points and they definitel...,Phillip To the extent that we can give Chair H...
2,2,Phillip I have a meeting tomorrow morning with...,Yes you can use this chart . Does it make sense ?
3,3,Phillip Which one should I do the x is half th...,I like the cedar t version better . Why don t ...
4,4,Phillip The Social Security and Medicare tax h...,Thanks


In [6]:
df=df.loc[:, ~df.columns.str.match('Unnamed')]
df.head(5)

,Received,Response
0,resumes of whom ?,The commercial support people that you and Hun...
1,Christy I read these points and they definitel...,Phillip To the extent that we can give Chair H...
2,Phillip I have a meeting tomorrow morning with...,Yes you can use this chart . Does it make sense ?
3,Phillip Which one should I do the x is half th...,I like the cedar t version better . Why don t ...
4,Phillip The Social Security and Medicare tax h...,Thanks


In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from torch.utils.data import Dataset, DataLoader

class EmailDataset(Dataset):
    def __init__(self, emails_df, tokenizer, max_length):
        self.emails = emails_df['Received']
        self.responses = emails_df['Response']
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.emails)

    def __getitem__(self, idx):
        email = str(self.emails[idx])
        response = str(self.responses[idx])

        input_text = email + response
        input_ids = self.tokenizer.encode(input_text, truncation=True, max_length=self.max_length, padding='max_length')

        return torch.tensor(input_ids)
    
    

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

In [9]:
config = GPT2Config.from_pretrained('gpt2')

In [10]:
model = GPT2LMHeadModel(config)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [12]:
batch_size = 8
num_epochs = 10
learning_rate = 2e-5

In [13]:
email_dataset = EmailDataset(df, tokenizer, max_length=512)
data_loader = DataLoader(email_dataset, batch_size=batch_size, shuffle=True)

In [14]:
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [16]:
# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for batch in data_loader:
        inputs = batch.to(device)

        # Shift the input tokens to the right
        inputs = torch.cat([inputs[:, -1:], inputs[:, :-1]], dim=-1)

        # Mask out the padding tokens
        attention_mask = (inputs != tokenizer.pad_token_id).long()

        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)

        outputs = model(inputs, attention_mask=attention_mask, labels=inputs)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch+1} - Average Loss: {avg_loss:.4f}")

# Save the trained model
model.save_pretrained('trained_model')
tokenizer.save_pretrained('trained_model')

Epoch 1 - Average Loss: 1.8080
Epoch 2 - Average Loss: 1.4338
Epoch 3 - Average Loss: 1.3032
Epoch 4 - Average Loss: 1.2058
Epoch 5 - Average Loss: 1.1278
Epoch 6 - Average Loss: 1.0618
Epoch 7 - Average Loss: 1.0028
Epoch 8 - Average Loss: 0.9474
Epoch 9 - Average Loss: 0.8953
Epoch 10 - Average Loss: 0.8461


('trained_model/tokenizer_config.json',
 'trained_model/special_tokens_map.json',
 'trained_model/vocab.json',
 'trained_model/merges.txt',
 'trained_model/added_tokens.json')